1.  **Melihat Gambaran Besar (*Look at the Big Picture*)**
    * **Tujuan Bisnis:** Model akan memprediksi harga rumah rata-rata di setiap distrik dan outputnya akan digunakan oleh sistem *Machine Learning* lain untuk menentukan kelayakan investasi di suatu area. Kesalahan estimasi harga manual saat ini bisa mencapai 20%.
    * **Framing Masalah:** Ini adalah tugas *supervised learning* karena ada data berlabel (harga rumah rata-rata). Ini juga merupakan tugas *regression*, khususnya *multiple regression* (menggunakan banyak fitur untuk prediksi) dan *univariate regression* (memprediksi satu nilai per distrik). Karena data tidak terus-menerus mengalir, *batch learning* cocok untuk masalah ini.
    * **Metrik Performa:** *Root Mean Square Error* (RMSE) adalah metrik umum untuk masalah regresi, memberikan bobot lebih tinggi pada kesalahan besar.
        $$RMSE(X,h)=\sqrt{\frac{1}{m}\sum_{i=1}^{m}(h(x^{(i)})-y^{(i)})^{2}}$$
        Di mana:
        * $m$: jumlah instansi dalam dataset.
        * $X^{(i)}$: vektor nilai fitur untuk instansi ke-$i$.
        * $y^{(i)}$: label (nilai output yang diinginkan) untuk instansi ke-$i$.
        * $h$: fungsi prediksi sistem (*hypothesis*).
        * $h(x^{(i)})$: nilai prediksi untuk instansi ke-$i$.
        RMSE adalah $l_2$ norm, sedangkan *Mean Absolute Error* (MAE) adalah $l_1$ norm. RMSE lebih sensitif terhadap *outlier* daripada MAE.
    * **Asumsi:** Penting untuk memeriksa asumsi di awal proyek. Dalam kasus ini, dipastikan bahwa sistem hilir membutuhkan harga aktual, bukan kategori.

2.  **Mendapatkan Data (*Get the Data*)**
    * Data yang digunakan adalah *California Housing Prices dataset* dari repositori StatLib, berdasarkan data sensus California tahun 1990.
    * **Membuat *Workspace*:** Mengatur lingkungan Python dengan menginstal modul seperti Jupyter, NumPy, pandas, Matplotlib, dan Scikit-Learn. Disarankan menggunakan *virtual environment* untuk menghindari konflik versi *library*.
        ```python
        # Contoh kode untuk membuat virtual environment (dari buku)
        # \$ export ML_PATH="\$HOME/ml"
        # \$ mkdir -p \$ML_PATH
        # \$ python3 -m pip install --user -U virtualenv
        # \$ cd \$ML_PAΤΗ
        # \$ python3 -m virtualenv my_env
        # \$ source my_env/bin/activate
        # \$ python3 -m pip install -U jupyter matplotlib numpy pandas scipy scikit-learn
        # \$ python3 -m ipykernel install --user --name python3
        # \$ jupyter notebook
        ```
    * **Mengunduh Data:** Membuat fungsi untuk mengunduh dan mengekstrak file `housing.tgz` yang berisi `housing.csv`.
        ```python
        import os
        import tarfile
        import urllib

        DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
        HOUSING_PATH = os.path.join("datasets", "housing")
        HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

        def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
            os.makedirs(housing_path, exist_ok=True)
            tgz_path = os.path.join(housing_path, "housing.tgz")
            urllib.request.urlretrieve(housing_url, tgz_path)
            housing_tgz = tarfile.open(tgz_path)
            housing_tgz.extractall(path=housing_path)
            housing_tgz.close()

        # fetch_housing_data() # Panggil fungsi ini untuk mengunduh data
        ```
    * **Melihat Struktur Data:** Menggunakan pandas untuk memuat data.
        ```python
        import pandas as pd

        def load_housing_data(housing_path=HOUSING_PATH):
            csv_path = os.path.join(housing_path, "housing.csv")
            return pd.read_csv(csv_path)

        housing = load_housing_data()
        housing.head() # Menampilkan 5 baris pertama
        housing.info() # Memberikan ringkasan data, termasuk jumlah baris, tipe atribut, dan nilai non-null
        housing["ocean_proximity"].value_counts() # Menghitung kategori untuk atribut 'ocean_proximity'
        housing.describe() # Menampilkan ringkasan statistik atribut numerik
        ```
        Dataset ini memiliki 20,640 instansi, dan atribut `total_bedrooms` memiliki 207 nilai yang hilang. Atribut `ocean_proximity` adalah kategorikal.
    * **Membuat *Test Set*:** Penting untuk membuat *test set* dan menyimpannya di awal untuk menghindari *data snooping bias*. Sekitar 20% data biasanya dialokasikan untuk *test set*.
        * **Sampling Acak:**
            ```python
            import numpy as np

            def split_train_test(data, test_ratio):
                shuffled_indices = np.random.permutation(len(data))
                test_set_size = int(len(data) * test_ratio)
                test_indices = shuffled_indices[:test_set_size]
                train_indices = shuffled_indices[test_set_size:]
                return data.iloc[train_indices], data.iloc[test_indices]

            # train_set, test_set = split_train_test(housing, 0.2)
            ```
            Kelemahan pendekatan ini adalah *test set* akan berbeda setiap kali program dijalankan. Solusi yang lebih baik adalah mengatur *random seed* atau menggunakan ID instansi untuk memastikan *split* yang stabil.
        * **Stratified Sampling:** Jika dataset tidak terlalu besar atau ada atribut penting, *stratified sampling* direkomendasikan untuk memastikan *test set* representatif terhadap populasi. Dalam kasus ini, median income digunakan sebagai dasar stratifikasi.
            ```python
            from sklearn.model_selection import StratifiedShuffleSplit

            housing["income_cat"] = pd.cut(housing["median_income"],
                                            bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                                            labels=[1, 2, 3, 4, 5])
            # housing["income_cat"].hist() # Untuk memvisualisasikan kategori pendapatan

            split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
            for train_index, test_index in split.split(housing, housing["income_cat"]):
                strat_train_set = housing.loc[train_index]
                strat_test_set = housing.loc[test_index]

            # strat_test_set["income_cat"].value_counts() / len(strat_test_set) # Memeriksa proporsi
            ```
            Setelah pembuatan *test set*, atribut `income_cat` harus dihapus agar data kembali ke keadaan semula.

3.  **Menjelajahi dan Memvisualisasikan Data (*Discover and Visualize the Data to Gain Insights*)**
    * Fokus pada *training set*.
    * **Visualisasi Geografis:** Menggunakan *scatterplot* untuk memvisualisasikan lokasi distrik berdasarkan *latitude* dan *longitude*. Mengatur `alpha` menjadi 0.1 membantu melihat area dengan kepadatan tinggi. Menambahkan informasi populasi (ukuran lingkaran) dan harga rumah (warna) ke *scatterplot* dapat memberikan wawasan lebih lanjut, menunjukkan bahwa harga terkait dengan lokasi dan kepadatan populasi.
    * **Mencari Korelasi:** Menghitung koefisien korelasi standar (*Pearson's r*) antar atribut.
        ```python
        corr_matrix = housing.corr()
        corr_matrix["median_house_value"].sort_values(ascending=False)
        ```
        Median income memiliki korelasi positif yang kuat dengan harga rumah median.
    * **Mengeksplorasi Kombinasi Atribut:** Membuat atribut baru yang mungkin lebih informatif, seperti `rooms_per_household`, `bedrooms_per_room`, dan `population_per_household`.
        ```python
        housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
        housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
        housing["population_per_household"] = housing["population"]/housing["households"]

        # Periksa lagi korelasi setelah menambahkan atribut baru
        # corr_matrix = housing.corr()
        # corr_matrix["median_house_value"].sort_values(ascending=False)
        ```
        Atribut `bedrooms_per_room` dan `rooms_per_household` terbukti lebih berkorelasi dengan harga rumah daripada atribut aslinya.

4.  **Mempersiapkan Data untuk Algoritma *Machine Learning* (*Prepare the Data for Machine Learning Algorithms*)**
    * Membuat salinan *training set* yang bersih dan memisahkan prediktor dari label.
    * **Pembersihan Data (*Data Cleaning*)**: Menangani nilai yang hilang. Ada tiga opsi:
        1.  Menghilangkan distrik yang relevan (`dropna()`)
        2.  Menghilangkan seluruh atribut (`drop()`)
        3.  Mengisi nilai yang hilang dengan nilai tertentu (nol, rata-rata, median, dll.) (`fillna()`).
        Scikit-Learn menyediakan `SimpleImputer` untuk menangani nilai yang hilang, biasanya dengan median untuk atribut numerik.
        ```python
        from sklearn.impute import SimpleImputer

        housing_num = housing.drop("ocean_proximity", axis=1) # Hanya atribut numerik
        imputer = SimpleImputer(strategy="median")
        imputer.fit(housing_num) # Menghitung median untuk setiap atribut
        X = imputer.transform(housing_num) # Mengisi nilai yang hilang
        housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)
        ```
    * **Menangani Atribut Teks dan Kategorikal:** Mengonversi atribut teks kategorikal menjadi angka.
        * **`OrdinalEncoder`:** Mengonversi kategori teks menjadi angka integer. Masalahnya adalah algoritma ML akan menganggap nilai yang berdekatan lebih mirip, yang tidak selalu benar.
            ```python
            from sklearn.preprocessing import OrdinalEncoder

            housing_cat = housing[["ocean_proximity"]]
            ordinal_encoder = OrdinalEncoder()
            housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
            # ordinal_encoder.categories_ # Melihat kategori yang dipelajari
            ```
        * **`OneHotEncoder`:** Solusi yang lebih baik adalah *one-hot encoding*, membuat atribut biner untuk setiap kategori. Ini menghasilkan *sparse matrix* yang efisien untuk menyimpan banyak nol.
            ```python
            from sklearn.preprocessing import OneHotEncoder

            cat_encoder = OneHotEncoder()
            housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
            # housing_cat_1hot.toarray() # Mengonversi ke dense NumPy array
            ```
    * ***Custom Transformers*:** Untuk operasi pembersihan kustom atau kombinasi atribut, Anda dapat membuat *transformer* sendiri dengan mengimplementasikan metode `fit()`, `transform()`, dan `fit_transform()` (dengan mewarisi `TransformerMixin` dan `BaseEstimator`).
        ```python
        from sklearn.base import BaseEstimator, TransformerMixin

        rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6 # Indeks kolom

        class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
            def __init__(self, add_bedrooms_per_room=True):
                self.add_bedrooms_per_room = add_bedrooms_per_room
            def fit(self, X, y=None):
                return self
            def transform(self, X):
                rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
                population_per_household = X[:, population_ix] / X[:, households_ix]
                if self.add_bedrooms_per_room:
                    bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
                    return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
                else:
                    return np.c_[X, rooms_per_household, population_per_household]

        # attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
        # housing_extra_attribs = attr_adder.transform(housing.values)
        ```
    * ***Feature Scaling*:** Sebagian besar algoritma *Machine Learning* tidak berkinerja baik jika atribut numerik memiliki skala yang sangat berbeda.
        * **Min-Max Scaling (Normalisasi):** Nilai diskalakan antara 0 dan 1. Digunakan `MinMaxScaler`.
        * **Standardisasi:** Mengurangkan nilai rata-rata dan membagi dengan standar deviasi, sehingga distribusi memiliki *unit variance*. Digunakan `StandardScaler`. Standardisasi kurang terpengaruh oleh *outlier*.
        Penting untuk hanya meng-*fit* *scaler* pada *training data*.
    * ***Transformation Pipelines*:** `Pipeline` Scikit-Learn membantu mengeksekusi urutan transformasi secara berurutan.
        ```python
        from sklearn.pipeline import Pipeline
        from sklearn.preprocessing import StandardScaler

        num_pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy="median")),
            ('attribs_adder', CombinedAttributesAdder()),
            ('std_scaler', StandardScaler()),
        ])

        # housing_num_tr = num_pipeline.fit_transform(housing_num)
        ```
        `ColumnTransformer` (di Scikit-Learn 0.20+) memungkinkan penerapan transformasi yang berbeda ke kolom yang berbeda.
        ```python
        from sklearn.compose import ColumnTransformer

        num_attribs = list(housing_num.columns) # Pastikan ini daftar nama kolom
        cat_attribs = ["ocean_proximity"]

        full_pipeline = ColumnTransformer([
            ("num", num_pipeline, num_attribs),
            ("cat", OneHotEncoder(), cat_attribs),
        ])

        housing_prepared = full_pipeline.fit_transform(housing)
        ```

5.  **Memilih dan Melatih Model (*Select and Train a Model*)**
    * **Melatih dan Mengevaluasi pada *Training Set*:**
        * **`LinearRegression`:**
            ```python
            from sklearn.linear_model import LinearRegression
            from sklearn.metrics import mean_squared_error

            lin_reg = LinearRegression()
            lin_reg.fit(housing_prepared, housing_labels)

            # Evaluasi
            # housing_predictions = lin_reg.predict(housing_prepared)
            # lin_mse = mean_squared_error(housing_labels, housing_predictions)
            # lin_rmse = np.sqrt(lin_mse)
            # print(f"Linear Regression RMSE: {lin_rmse}")
            ```
            RMSE yang tinggi menunjukkan *underfitting*.
        * **`DecisionTreeRegressor`:** Model yang lebih kompleks.
            ```python
            from sklearn.tree import DecisionTreeRegressor

            tree_reg = DecisionTreeRegressor()
            tree_reg.fit(housing_prepared, housing_labels)

            # Evaluasi
            # housing_predictions = tree_reg.predict(housing_prepared)
            # tree_mse = mean_squared_error(housing_labels, housing_predictions)
            # tree_rmse = np.sqrt(tree_mse)
            # print(f"Decision Tree RMSE: {tree_rmse}") # Seringkali 0.0, menunjukkan overfitting parah
            ```
            RMSE 0.0 pada *training set* mengindikasikan *overfitting* yang parah.
    * **Evaluasi yang Lebih Baik Menggunakan *Cross-Validation*:**
        `K-fold cross-validation` membagi *training set* menjadi beberapa *fold* dan melatih serta mengevaluasi model berkali-kali, menggunakan *fold* yang berbeda untuk evaluasi setiap kali.
        ```python
        from sklearn.model_selection import cross_val_score

        def display_scores(scores):
            print("Scores:", scores)
            print("Mean:", scores.mean())
            print("Standard deviation:", scores.std())

        # Untuk DecisionTreeRegressor
        scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                                 scoring="neg_mean_squared_error", cv=10)
        tree_rmse_scores = np.sqrt(-scores)
        display_scores(tree_rmse_scores)

        # Untuk LinearRegression
        lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                                     scoring="neg_mean_squared_error", cv=10)
        lin_rmse_scores = np.sqrt(-lin_scores)
        display_scores(lin_rmse_scores)
        ```
        Hasil *cross-validation* menunjukkan bahwa `DecisionTreeRegressor` *overfit* dan bahkan berkinerja lebih buruk daripada `LinearRegression`.
        * **`RandomForestRegressor`:** Model *Ensemble Learning* yang seringkali memberikan hasil yang lebih baik.
            ```python
            from sklearn.ensemble import RandomForestRegressor

            forest_reg = RandomForestRegressor(random_state=42) # Tambahkan random_state agar reproducible
            forest_reg.fit(housing_prepared, housing_labels)

            # Evaluasi dengan cross-validation
            forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                            scoring="neg_mean_squared_error", cv=10)
            forest_rmse_scores = np.sqrt(-forest_scores)
            display_scores(forest_rmse_scores)
            ```
            `RandomForestRegressor` menunjukkan hasil yang jauh lebih baik, tetapi masih ada indikasi *overfitting*. Disarankan untuk mencoba berbagai model lain dan menyimpan model yang menjanjikan.

6.  **Menyesuaikan Model (*Fine-Tune Your Model*)**
    * **`Grid Search`:** `GridSearchCV` Scikit-Learn secara otomatis mencari kombinasi *hyperparameter* terbaik menggunakan *cross-validation*.
        ```python
        from sklearn.model_selection import GridSearchCV

        param_grid = [
            {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
            {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
        ]

        forest_reg = RandomForestRegressor(random_state=42)
        grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                                   scoring='neg_mean_squared_error',
                                   return_train_score=True)

        grid_search.fit(housing_prepared, housing_labels)

        # grid_search.best_params_ # Mendapatkan hyperparameter terbaik
        # grid_search.best_estimator_ # Mendapatkan estimator terbaik
        # cvres = grid_search.cv_results_
        # for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
        #    print(np.sqrt(-mean_score), params)
        ```
        `GridSearchCV` akan melatih model berkali-kali untuk setiap kombinasi *hyperparameter*.
    * **`Randomized Search`:** Ketika ruang pencarian *hyperparameter* besar, `RandomizedSearchCV` lebih disukai. Ia mengevaluasi sejumlah kombinasi acak.

7.  **Menganalisis Model Terbaik dan Kesalahannya (*Analyze the Best Models and Their Errors*)**
    * Mengevaluasi `feature_importances_` dari `RandomForestRegressor` untuk memahami pentingnya setiap atribut dalam membuat prediksi yang akurat.
        ```python
        # feature_importances = grid_search.best_estimator_.feature_importances_
        # extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
        # cat_encoder = full_pipeline.named_transformers_["cat"]
        # cat_one_hot_attribs = list(cat_encoder.categories_[0])
        # attributes = num_attribs + extra_attribs + cat_one_hot_attribs
        # sorted(zip(feature_importances, attributes), reverse=True)
        ```
        Informasi ini dapat digunakan untuk menghilangkan fitur yang kurang berguna.

8.  **Mengevaluasi Sistem pada *Test Set* (*Evaluate Your System on the Test Set*)**
    * Setelah *fine-tuning* selesai, evaluasi model akhir pada *test set* menggunakan metode `transform()` pada *pipeline* data.
        ```python
        final_model = grid_search.best_estimator_

        X_test = strat_test_set.drop("median_house_value", axis=1)
        y_test = strat_test_set["median_house_value"].copy()

        X_test_prepared = full_pipeline.transform(X_test)
        final_predictions = final_model.predict(X_test_prepared)

        final_mse = mean_squared_error(y_test, final_predictions)
        final_rmse = np.sqrt(final_mse) # Sekitar 47,730.2

        # Menghitung interval kepercayaan 95%
        # from scipy import stats
        # confidence = 0.95
        # squared_errors = (final_predictions - y_test) ** 2
        # np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
        #                         loc=squared_errors.mean(),
        #                         scale=stats.sem(squared_errors)))
        ```
    * Jika performa pada *test set* sedikit lebih buruk dari *cross-validation*, ini normal karena model mungkin terlalu disesuaikan dengan data validasi.

9.  **Meluncurkan, Memantau, dan Memelihara Sistem (*Launch, Monitor, and Maintain Your System*)**
    * **Penyebaran (*Deployment*):** Model yang telah dilatih dapat disimpan (misalnya, menggunakan `joblib`) dan dimuat di lingkungan produksi. Ini bisa berupa aplikasi web lokal atau layanan web khusus (misalnya, REST API). Penyebaran ke *cloud* seperti Google Cloud AI Platform juga merupakan pilihan populer.
    * **Pemantauan (*Monitoring*):** Penting untuk memantau performa model secara langsung dan memicu peringatan jika performanya menurun. Model cenderung "membusuk" seiring waktu karena perubahan data. Pemantauan juga harus mencakup kualitas data input.
    * **Pemeliharaan (*Maintenance*):** Proses pengumpulan data baru, pelabelan, pelatihan ulang model, dan penyebaran model baru harus otomatis jika data terus berkembang. Penting juga untuk memiliki *backup* model dan dataset untuk *rollback* jika terjadi kegagalan.

Singkatnya, Bab 2 menekankan pentingnya proses *end-to-end* yang sistematis dalam proyek *Machine Learning*, mulai dari pemahaman masalah bisnis, persiapan data yang cermat, pemilihan dan pelatihan model, hingga *fine-tuning* dan aspek pemeliharaan sistem di lingkungan produksi. Ini juga memperkenalkan berbagai alat dan teknik penting dari Scikit-Learn.